## Introduction

Linked lists are the best and simplest example of a dynamic data structure that uses pointers for its implementation.

Linked lists are arrays that can grow and shrink as needed, from any point of that array.

Linked lists have a few advantages over plain arrays:
1. Items can be added or removed from the middle of the list
2. There is no need to define an initial size

Meanwhile, linked lists have a few disadvantages:
1. There is no random access - it is impossible to reach the ```nth``` item in the array without iterating over all items up until that item.
2. Dynamic memory allocation and pointers required - complicates the code.
3. Much larger overhead over arrays, since linked list items are dynamically allocated and each item in the list must store an additional pointer.

## What is a linked list?

A linked list is a set of dynamically allocated nodes, arranged in such a way that each node contains one value and one ```pointer```. The ```pointer``` always points to the next item of the list. The last item's ```pointer``` is ```NULL```.

A linked list is held using a ```local pointer``` variable which points to the first item of the list. If that ```pointer``` is also ```NULL```, then the list is considered to be empty.

    ------------------------------              ------------------------------
    |              |             |            \ |              |             |
    |     DATA     |     NEXT    |--------------|     DATA     |     NEXT    |
    |              |             |            / |              |             |
    ------------------------------              ------------------------------

Let's define a linked list node, Name the node type ```node_t```

In [28]:
typedef struct Node {
    int val;
    struct Node * next;
} node_t;

Now we can use the ```nodes```. Let's create a local variable which points to the first item of the list (called ```head```). Set the ```node_t.value``` be 1, and the ```node_t.next``` to be empty. Note we should always check if the ```malloc ``` returned a ```NULL``` or not.

In [29]:
node_t * head = NULL;
head = (node_t *) malloc(sizeof(node_t));

if (head == NULL) {
    return 1;
}

head->val = 1;
head->next = NULL;

To add variables to the end of the list, we can just continue advancing to the next pointer:

In [30]:
node_t * head = NULL;
head = (node_t *) malloc(sizeof(node_t));
head->val = 1;
head->next = (node_t *) malloc(sizeof(node_t));
head->next->val = 2;
head->next->next = NULL;

## Iterate over a list

Iterate over the list and print, use ```current``` pointer to keep track of the node we are currently concerning. After printing, ```current``` is set to the ```next``` node. We repeat this process until ```node-next == NULL```. (end of the list)

### Iterate to print the list

In [31]:
void print_list(node_t * head) {
    node_t * current = head;

    while (current != NULL) {
        printf("Current value: %d, Next node address: %p\n", current->val, current->next);
        current = current->next;
    }
}

### Adding an item to the end of the list
To add to last, we start from the head and then in each step, we advance the pointer to the next item in the list

In [32]:
void push_last(node_t * head, int val) {
    node_t * current = head;
    while (current->next != NULL) {
        current = current -> next;
    }
    
    /* reached the last node */
    current->next = (node_t *) malloc (sizeof(node_t));
    current->next->val = val;
    current->next->next = NULL;
}

In [33]:
push_last(head, 3);
push_last(head, 9);
print_list(head);

Current value: 1, Next node address: 0x5562840e9530
Current value: 2, Next node address: 0x5562846ce350
Current value: 3, Next node address: 0x556284305500
Current value: 9, Next node address: (nil)


### Remove the last item of the list
Removing the last item requires us to traverse the linked list, but track two items in one iteration. If the ```node-next-next == NULL```, we stop and remove the last item.

In [34]:
int remove_last(node_t * head) {
    int last_val = 0;
    
    /* if there is only one item in the list, remove it */
    if (head->next == NULL) {
        last_val = head->val;
        free(head);
        return last_val;
    }
    
    /* traverse the linked list to the second to the last node */
    node_t * current = head;
    while(current->next->next != NULL) {
        current = current -> next;
    }
    
    last_val = current->next->val;
    free(current->next);
    current->next = NULL;
    return last_val;
}

In [35]:
int last_removed = remove_last(head);
printf("Removed value: %d\n\n", last_removed);
print_list(head);

Removed value: 9

Current value: 1, Next node address: 0x5562840e9530
Current value: 2, Next node address: 0x5562846ce350
Current value: 3, Next node address: (nil)


### Adding item to the beginning of the list

To add to the beginning, we need to: 
1. Create a new item and set the value.
2. Link the item to point to the head of the list.
3. Set the ```head``` of the list be the new item.

As we need to know the address of the first node, using ```ponter to pointer``` is necessary.

In [36]:
void push_first(node_t ** head, int val) {
    node_t * new_node = (node_t *) malloc (sizeof(node_t));
    new_node->val = val;
    new_node->next = *head;
    *head = new_node;
}

In [37]:
push_first(&head, 8);
push_first(&head, 7);
print_list(head);

Current value: 7, Next node address: 0x5562840eb570
Current value: 8, Next node address: 0x5562846d3e70
Current value: 1, Next node address: 0x5562840e9530
Current value: 2, Next node address: 0x5562846ce350
Current value: 3, Next node address: (nil)


### Remove the first item from a list
To pop a variable, we need to:

1. Take the address of the next item and save it in a temp variable.
2. Free the head item.
3. Set the head to be the next item that we have sored 

In [38]:
int remove_first(node_t ** head) {
    int first_val = 0;
    node_t * next_node = NULL;
    
    if (*head == NULL) {
        return -1;
    }
    
    next_node = (*head) -> next;
    
    first_val = (*head) -> val;
    free(*head);

    *head = next_node;
    
    return first_val;
}

In [39]:
int firstremoved = remove_first(&head);
print_list(head)

Current value: 8, Next node address: 0x5562846d3e70
Current value: 1, Next node address: 0x5562840e9530
Current value: 2, Next node address: 0x5562846ce350
Current value: 3, Next node address: (nil)


### Remove a specific item (by index)

To remove a specific item from the list, either by its index from the beginning of the list or by its value. Traversing the entire linked list might be necessary. The algorithm could be:
1. Iterate the node before we wish to delete.
2. Save the node we wish to delete in a temporary pointer.
3. Set the previous node's next pointer to point to the node after the node to be deleted.
4. Delete the node using the temporary pointer.

There are a few edge cases that should be taken care of: 

In [40]:
int remove_by_index(node_t ** head, int n) {
    int i = 0;
    int removed_val = 0;
    node_t * current = *head;
    node_t * temp_node = NULL;
    
    if (n==0) {
        return remove_first(head);
    }
    
    for (i = 0; i < n-1; i++) {
        if (current->next == NULL) {
            return -1;
        }
        current = current -> next;
    }
    
    if (current->next == NULL) {
        return remove_last(*head);
    }
    
    temp_node = current->next;
    removed_val = temp_node->val;
    current->next = temp_node->next;
    free(temp_node);
    
    return removed_val;
} 

In [41]:
int removed = remove_by_index(&head, 2);
print_list(head);

Current value: 8, Next node address: 0x5562846d3e70
Current value: 1, Next node address: 0x5562846ce350
Current value: 3, Next node address: (nil)


### Add a specific item (by index)

In [42]:
void add_by_index(node_t ** head, int n, int val) {
    int i = 0;
    node_t * current = *head;
    node_t * added_node = NULL;
    
    if (n==0) {
        push_first(head, val);
    }
    
    for (i = 0; i < n-1; i++) {
        if (current->next == NULL) {
            return -1;
        }
        current = current -> next;
    }
    
    if (current->next == NULL) {
        return push_last(*head, val);
    }
    
    added_node = (node_t *) malloc (sizeof(node_t));
    added_node->val = val;
    added_node->next = current->next;
    current->next = added_node;
} 

In [43]:
add_by_index(&head, 1, 200);
print_list(head);

Current value: 8, Next node address: 0x5562843c97b0
Current value: 200, Next node address: 0x5562846d3e70
Current value: 1, Next node address: 0x5562846ce350
Current value: 3, Next node address: (nil)


### Add a specific item (by value)

In [44]:
void add_by_value(node_t ** head, int val) {
    node_t * current = *head;
    node_t * added_node = NULL;
    
    while (current->val != val) {
        if (current->next == NULL) {
            return -1;
        }
        current = current -> next;
    }
    
    added_node = (node_t *) malloc (sizeof(node_t));
    added_node->val = val;
    added_node->next = current->next;
    current->next = added_node;
}

In [45]:
print_list(head);

Current value: 8, Next node address: 0x5562843c97b0
Current value: 200, Next node address: 0x5562846d3e70
Current value: 1, Next node address: 0x5562846ce350
Current value: 3, Next node address: (nil)


In [46]:
add_by_value(&head, 8);
print_list(head);

Current value: 8, Next node address: 0x556284141320
Current value: 8, Next node address: 0x5562843c97b0
Current value: 200, Next node address: 0x5562846d3e70
Current value: 1, Next node address: 0x5562846ce350
Current value: 3, Next node address: (nil)


### Remove a specific item (by value)

In [47]:
int remove_by_value(node_t ** head, int val) {
    int removed_val = 0;
    node_t * current = *head;
    node_t * prev_node = NULL;
    
    while (current->val != val) {
        if (current->next == NULL) {
            return -1;
        }
        prev_node = current;
        current = current -> next;
    }
    
    if (prev_node == NULL) {
        return remove_first(head);
    }
    removed_val = current->val;
    prev_node->next = current->next;
    free(current);
    
    return removed_val;
}

In [48]:
remove_by_value(&head, 8);
print_list(head);

Current value: 8, Next node address: 0x5562843c97b0
Current value: 200, Next node address: 0x5562846d3e70
Current value: 1, Next node address: 0x5562846ce350
Current value: 3, Next node address: (nil)
